In [1]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import logging ### to monitor the code
from bs4 import BeautifulSoup
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
from urllib.request import urlopen 
import io
import getpass
import time

import xml.etree.cElementTree as ET
from urllib.request import urlopen

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

!pip install PyGithub
from github import Github
!pip install sddk
import sddk

     |████████████████████████████████| 215kB 2.8MB/s 


In [2]:
conf = sddk.configure_session_and_url("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


# Get paths and filenames for a GitHub repo...

In [46]:
### make a github authorization
g = Github("kasev", getpass.getpass("github password: "))

github password: ··········


In [0]:
### GET PATHS+NAMES OF ALL FILES OF INTEREST WITHIN A GITHUB REPO
repo = g.get_user("PerseusDL").get_repo("canonical-greekLit" )
file_paths = []
contents = repo.get_contents("data")
while contents:
  file_content = contents.pop(0)
  if file_content.type == "dir":
    contents.extend(repo.get_contents(file_content.path))
  else:
    try:
      path_and_name = file_content.path
      if ".xml" in str(path_and_name):
        if "perseus-grc" in str(path_and_name):
          print(path_and_name)
          file_paths.append(str(path_and_name))
    except: 
      pass

In [50]:
sddk.write_file("SDAM_data/OGL/perseus_paths.json", file_paths, conf)

A file with the same name ("perseus_paths.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'list'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/OGL/perseus_paths.json"


In [0]:
repo = g.get_user("OpenGreekAndLatin").get_repo("First1KGreek" )
file_paths_ogl = []
contents = repo.get_contents("data")
while contents:
  file_content = contents.pop(0)
  if file_content.type == "dir":
    contents.extend(repo.get_contents(file_content.path))
  else:
    try:
      path_and_name = file_content.path
      if ".xml" in str(path_and_name):
        if "__cts__" not in str(path_and_name):    
          file_paths_ogl.append(str(path_and_name))
    except: 
      pass

In [52]:
file_paths_ogl[:10]

['data/ggm0001/ggm001/ggm0001.ggm001.1st1K-grc1.xml',
 'data/heb0001/heb010/heb0001.heb010.1st1K-eng1.xml',
 'data/ogl0001/ogl001/ogl0001.ogl001.1st1K-grc1.xml',
 'data/stoa0033a/tlg028/stoa0033a.tlg028.1st1K-grc1.xml',
 'data/stoa0033a/tlg043/stoa0033a.tlg043.1st1K-grc1.xml',
 'data/stoa0121/stoa001/stoa0121.stoa001.opp-grc1.xml',
 'data/stoa0146d/stoa001/stoa0146d.stoa001.opp-grc1.xml',
 'data/stoa0146d/stoa001/stoa0146d.stoa001.opp-lat1.xml',
 'data/tlg0011/tlg003/tlg0011.tlg003.1st1K-grc1.xml',
 'data/tlg0015/tlg001/tlg0015.tlg001.opp-grc1.xml']

In [53]:
sddk.write_file("SDAM_data/OGL/ogl_paths.json", file_paths_ogl, conf)

A file with the same name ("ogl_paths.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'list'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/OGL/ogl_paths.json"


## Testing with individual texts from xml files

In [0]:
response = requests.get('https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0086/tlg009/tlg0086.tlg009.perseus-grc1.xml')
soup = BeautifulSoup(response.text, "html.parser")

In [56]:
soup.titlestmt.title.get_text()

'Eudemian Ethics (Greek). Machine readable text'

In [21]:
all_p = soup.body.find_all("p")
title = soup.titlestmt.title.get_text()
raw_text = []
for p in all_p:
  raw_text.append(p.get_text())
raw_text_str = " ".join(raw_text)
raw_text_str

'\n\n\n ὁ μὲν ἐν Δήλῳ παρὰ τῷ θεῷ τὴν αὑτοῦ γνώμην ἀποφηνάμενος συνέγραψεν ἐπὶ τὸ προπύλαιον τοῦ Λητῴου, διελὼν οὐχ ὑπάρχοντα πάντα τῷ αὐτῷ, τό τε ἀγαθὸν καὶ τὸ καλὸν καὶ τὸ ἡδύ, ποιήσας \n\n\nκάλλιστον τὸ δικαιότατον, λῷστον δʼ ὑγιαίνειν·,\nπάντων ἥδιστον δʼ οὗ τις ἐρᾷ τὸ τυχεῖν·\n\nTheog. 255f\n ἡμεῖς δʼ αὐτῷ μὴ συγχωρῶμεν. ἡ γὰρ εὐδαιμονία κάλλιστον καὶ ἄριστον ἁπάντων οὖσα ἥδιστον ἐστίν.  πολλῶν δʼ ὄντων θεωρημάτων ἃ περὶ ἕκαστον πρᾶγμα καὶ  περὶ ἑκάστην φύσιν ἀπορίαν ἔχει καὶ δεῖται σκέψεως, τὰ μὲν αὐτῶν συντείνει πρὸς τὸ γνῶναι μόνον, τὰ δὲ καὶ περὶ τὰς κτήσεις καὶ περὶ τὰς πράξεις τοῦ πράγματος. ὅσα μὲν οὖν ἔχει φιλοσοφίαν μόνον θεωρητικήν, λεκτέον κατὰ τὸν ἐπιβάλλοντα καιρόν, ὅ τι περ οἰκεῖον ἦν τῇ μεθόδῳ· πρῶτον δὲ  σκεπτέον ἐν τίνι τὸ εὖ ζῆν καὶ πῶς κτητόν, πότερον φύσει γίγνονται πάντες εὐδαίμονες οἱ τυγχάνοντες ταύτης τῆς προσηγορίας, ὥσπερ μεγάλοι καὶ μικροὶ καὶ τὴν χροιὰν διαφέροντες, ἢ διὰ μαθήσεως, ὡς οὔσης ἐπιστήμης τινὸς τῆς εὐδαιμονίας, ἢ διά τινος ἀσκήσεως ʽπολλὰ γὰ

## Testing with a sample

In [0]:
### start with the paths
paths_ogl = sddk.read_file("SDAM_data/OGL/ogl_paths.json", "list", conf)
paths_perseus = sddk.read_file("SDAM_data/OGL/perseus_paths.json", "list", conf)

In [67]:
## testing
%%time
data = []
for filepath in random.choices(paths_perseus, k=10):
  response = requests.get('https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/' + filepath)
  soup = BeautifulSoup(response.text, "html.parser")
  try:
    string = " ".join(soup.body.text.split())
  except:
    string = "" 
  try:
    author = soup.author.text
  except:
    author = None
  try:
    title = soup.titlestmt.title.get_text()
  except:
    title = None
  data.append([filepath.rpartition("/")[2], author, title, string])
print(len(data))

10
CPU times: user 697 ms, sys: 11 ms, total: 708 ms
Wall time: 3.77 s


In [68]:
### make a short look
data

[['tlg0014.tlg050.perseus-grc2.xml',
  'Demosthenes',
  'πρὸς Πολυκλέα περὶ τοῦ ἐπιτριηραρχήματος',
  'τοῖς τοιούτοις τῶν ἀγώνων, ὦ ἄνδρες δικασταί, καὶ τοὺς διαγνωσομένους προσήκει μάλιστα προσέχειν τὸν νοῦν. οὐ γὰρ ἐμὸς καὶ Πολυκλέους ἴδιός ἐστιν ὁ ἀγὼν μόνον, ἀλλὰ καὶ τῆς πόλεως κοινός. ὧν γὰρ τὰ μὲν ἐγκλήματα ἴδιά ἐστιν, αἱ δὲ βλάβαι κοιναί, πῶς οὐχ ὑπὲρ τούτων εἰκός ἐστιν ἀκούσαντας ὑμᾶς ὀρθῶς διαγνῶναι; εἰ μὲν γὰρ περὶ ἄλλου τινὸς συμβολαίου ἐγὼ διαφερόμενος πρὸς Πολυκλέα εἰσῄειν εἰς ὑμᾶς, ἐμὸς ἂν ἦν καὶ Πολυκλέους ὁ ἀγών· νῦν δὲ περί τε διαδοχῆς νεώς ἐστιν ὁ λόγος καὶ ἐπιτριηραρχήματος πέντε μηνῶν καὶ ἓξ ἡμερῶν ἀνηλωμένου, καὶ περὶ τῶν νόμων, πότερα κύριοί εἰσιν ἢ οὔ. ἀναγκαῖον δή μοι δοκεῖ εἶναι ἅπαντα ἐξ ἀρχῆς διηγήσασθαι πρὸς ὑμᾶς. καὶ πρὸς θεῶν, ἄνδρες δικασταί, δέομαι ὑμῶν, μή με ἡγήσησθε ἀδολεσχεῖν, ἐὰν διὰ μακροτέρων διηγῶμαι τά τε ἀναλώματα καὶ τὰς πράξεις, ὡς ἐν καιρῷ τε ἕκασται καὶ χρήσιμοι τῇ πόλει ἐδιακονήθησαν. εἰ μὲν γάρ τις ἔχει με ἐπιδεῖξαι ὡς ψεύδομαι, ἀναστὰς ἐ

## Extraction of the whole dataset

In [75]:
## actual extraction
%%time
data = []
for filepath in paths_perseus:
  try:
    response = requests.get('https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/' + filepath)  
    soup = BeautifulSoup(response.text, "html.parser")
  except:
    time.sleep(1)
    response = requests.get('https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/' + filepath)  
    soup = BeautifulSoup(response.text, "html.parser")
  try:
    string = " ".join(soup.text.body.split())
  except:
    try:
      string = " ".join(soup.body.text.split())
    except:
      string = ""
  try:
    author = soup.author.text
  except:
    author = None
  try:
    title = soup.titlestmt.title.get_text()
  except:
    title = None
  data.append([filepath.rpartition("/")[2], author, title, string])
print(len(data))

793
CPU times: user 54.7 s, sys: 741 ms, total: 55.5 s
Wall time: 3min 59s


In [76]:
data_perseus = pd.DataFrame(data, columns=["filename", "author", "title", "string"])
data_perseus.head(5)

filename  ...                                             string
0  tlg0001.tlg001.perseus-grc2.xml  ...  ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...
1  tlg0003.tlg001.perseus-grc2.xml  ...  Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν ...
2  tlg0004.tlg001.perseus-grc1.xml  ...  ΒΙΩΝ ΚΑΙ ΓΝΩΜΩΝ ΤΩΝ ΕΝ ΦΙΛΟΣΟΦΙΑΙ ΕΥΔΟΚΙΜΗΣΑΝΤ...
3  tlg0005.tlg001.perseus-grc1.xml  ...  Θύρσις ἢ ᾠδή Θύρσις ̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ...
4  tlg0005.tlg002.perseus-grc1.xml  ...  α παλ ϝι.336 τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυ...

[5 rows x 4 columns]

In [77]:
data_perseus[data_perseus["string"] == ""]

filename author title string
5    tlg0006.tlg007.perseus-grc1.xml   None  None       
6    tlg0006.tlg008.perseus-grc1.xml   None  None       
8    tlg0006.tlg010.perseus-grc1.xml   None  None       
9    tlg0006.tlg011.perseus-grc1.xml   None  None       
10   tlg0006.tlg012.perseus-grc1.xml   None  None       
11   tlg0006.tlg013.perseus-grc1.xml   None  None       
12   tlg0006.tlg014.perseus-grc1.xml   None  None       
13   tlg0006.tlg015.perseus-grc1.xml   None  None       
14   tlg0006.tlg016.perseus-grc1.xml   None  None       
129  tlg0007.tlg098.perseus-grc1.xml   None  None       
131  tlg0007.tlg099.perseus-grc1.xml   None  None       
133  tlg0007.tlg100.perseus-grc1.xml   None  None       
135  tlg0007.tlg101.perseus-grc1.xml   None  None       
136  tlg0007.tlg102.perseus-grc1.xml   None  None       
137  tlg0007.tlg103.perseus-grc1.xml   None  None       
138  tlg0007.tlg104.perseus-grc1.xml   None  None       
139  tlg0007.tlg105.perseus-grc1.xml   None  None       
140  tlg0007.tlg106.perseus-grc1.xml   None  None       
141  tlg0007.tlg107.perseus-grc1.xml   None  None       
142  tlg0007.tlg108.perseus-grc1.xml   None  None       
143  tlg0007.tlg109.perseus-grc1.xml   None  None       
144  tlg0007.tlg110.perseus-grc1.xml   None  None       
145  tlg0007.tlg111.perseus-grc1.xml   None  None       
146  tlg0007.tlg112.perseus-grc1.xml   None  None       
147  tlg0007.tlg113.perseus-grc1.xml   None  None       
148  tlg0007.tlg114.perseus-grc1.xml   None  None       
149  tlg0007.tlg115.perseus-grc1.xml   None  None       
150  tlg0007.tlg116.perseus-grc1.xml   None  None       
151  tlg0007.tlg117.perseus-grc1.xml   None  None       
153  tlg0007.tlg118.perseus-grc1.xml   None  None       
154  tlg0007.tlg119.perseus-grc1.xml   None  None       
155  tlg0007.tlg120.perseus-grc1.xml   None  None       
156  tlg0007.tlg121.perseus-grc1.xml   None  None       
157  tlg0007.tlg122.perseus-grc1.xml   None  None       
158  tlg0007.tlg123.perseus-grc1.xml   None  None       
159  tlg0007.tlg125.perseus-grc1.xml   None  None       
160  tlg0007.tlg126.perseus-grc1.xml   None  None       
717  tlg0557.tlg001.perseus-grc1.xml   None  None

In [78]:
sddk.write_file("SDAM_data/OGL/data_perseus.json", data_perseus, conf)

A file with the same name ("data_perseus.json") already exists in this location.


KeyboardInterrupt: ignored

In [0]:
## ogl 
%%time
data = []
for filepath in paths_ogl:
  response = requests.get('https://raw.githubusercontent.com/OpenGreekAndLatin/First1KGreek/master/' + filepath)
  soup = BeautifulSoup(response.text, "html.parser")
   try:
    string = " ".join(soup.text.body.split())
  except:
    try:
      string = " ".join(soup.body.text.split())
    except:
      string = ""
  try:
    author = soup.author.text
  except:
    author = None
  try:
    title = ""
    for t in soup.find_all("title"):
      title = title + "; " + t.get_text()
    title = title[2:]
  except:
    title = None
  data.append([filepath.rpartition("/")[2], author, title, string])
print(len(data))

In [0]:
data_ogl = pd.DataFrame(data, columns=["filename", "author", "title", "string"])
data_ogl.head(5)

In [0]:
sddk.write_file("SDAM_data/OGL/data_ogl.json", data_perseus, conf)

# Read and Merge the Data

In [0]:
### READ THE DATA FROM SCIENCEDATA.DK
### it takes quite long, for some reason...
%%time

data_perseus = pd.DataFrame(s.get(sddk_url + "SDAM_data/OGL/data_perseus.json").json())
data_ogl= pd.DataFrame(s.get(sddk_url + "SDAM_data/OGL/data_ogl.json").json())

First, we have to explore whether there is an overlap in file names between the two corpora

In [0]:
### produce a list of filenames in the perseus dataset
filenames_perseus = data_perseus.filename.unique().tolist()
print(filenames_perseus[:10])

['tlg0001.tlg001.perseus-grc2.xml', 'tlg0003.tlg001.perseus-grc2.xml', 'tlg0004.tlg001.perseus-grc1.xml', 'tlg0005.tlg001.perseus-grc1.xml', 'tlg0005.tlg002.perseus-grc1.xml', 'tlg0006.tlg007.perseus-grc1.xml', 'tlg0006.tlg008.perseus-grc1.xml', 'tlg0006.tlg009.perseus-grc2.xml', 'tlg0006.tlg010.perseus-grc1.xml', 'tlg0006.tlg011.perseus-grc1.xml']


In [0]:
works_perseus = [filename.rpartition(".")[0].rpartition(".")[0] for filename in filenames_perseus]
print(works_perseus)

['tlg0001.tlg001', 'tlg0003.tlg001', 'tlg0004.tlg001', 'tlg0005.tlg001', 'tlg0005.tlg002', 'tlg0006.tlg007', 'tlg0006.tlg008', 'tlg0006.tlg009', 'tlg0006.tlg010', 'tlg0006.tlg011', 'tlg0006.tlg012', 'tlg0006.tlg013', 'tlg0006.tlg014', 'tlg0006.tlg015', 'tlg0006.tlg016', 'tlg0006.tlg017', 'tlg0006.tlg018', 'tlg0006.tlg019', 'tlg0007.tlg001', 'tlg0007.tlg002', 'tlg0007.tlg003', 'tlg0007.tlg004', 'tlg0007.tlg005', 'tlg0007.tlg006', 'tlg0007.tlg007', 'tlg0007.tlg008', 'tlg0007.tlg009', 'tlg0007.tlg010', 'tlg0007.tlg011', 'tlg0007.tlg012', 'tlg0007.tlg013', 'tlg0007.tlg014', 'tlg0007.tlg015', 'tlg0007.tlg016', 'tlg0007.tlg017', 'tlg0007.tlg018', 'tlg0007.tlg019', 'tlg0007.tlg020', 'tlg0007.tlg021', 'tlg0007.tlg022', 'tlg0007.tlg023', 'tlg0007.tlg024', 'tlg0007.tlg025', 'tlg0007.tlg026', 'tlg0007.tlg027', 'tlg0007.tlg028', 'tlg0007.tlg029', 'tlg0007.tlg030', 'tlg0007.tlg031', 'tlg0007.tlg032', 'tlg0007.tlg033', 'tlg0007.tlg034', 'tlg0007.tlg035', 'tlg0007.tlg036', 'tlg0007.tlg037', 'tlg0007.

In [0]:
def check_filename(row_and_column):
  perseus = False
  for work in works_perseus:
    if row_and_column.startswith(work):
      perseus = True
      continue
  return perseus

In [0]:
data_ogl["in_perseus"] = data_ogl.apply(lambda row: check_filename(row["filename"]), axis=1)
### to filter for values not in perseus
data_ogl = data_ogl[data_ogl["in_perseus"]==False]

In [0]:
data_ogl.drop(columns=["in_perseus"], inplace=True)
len(data_ogl)

870

In [0]:
### finally, merge the two datasets (perseus + ogl)
data_merged_raw = pd.concat([data_ogl, data_perseus])

In [0]:
### how many files we
len(data_merged_raw)

1663

In [0]:
data_merged_raw.reset_index(inplace=True)
data_merged_raw.drop(columns=["index"], inplace=True)